In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data_backup = pd.read_csv('../csvs/deliveries.csv')
data = data_backup.copy()
data.columns

Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'ball',
       'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler',
       'runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'penalty', 'wicket_type', 'player_dismissed', 'other_wicket_type',
       'other_player_dismissed'],
      dtype='object')

In [3]:
original_df = data

original_df['start_date'] = pd.to_datetime(original_df['start_date'])

grouped_df = original_df.groupby(['match_id', 'innings', 'venue'])

result_df = grouped_df.agg({
    'batting_team': 'first',
    'bowling_team': 'first',
    'wides': 'sum',
    'noballs': 'sum',
    'byes': 'sum',
    'legbyes': 'sum',
    'runs_off_bat': 'sum',
    'ball': 'max',
    'wicket_type': lambda x: x.notnull().sum(),
    'start_date': 'first'
}).reset_index()

result_df.columns = ['match_id', 'innings', 'venue', 'batting_team', 'bowling_team',
                      'total_wides', 'total_noballs', 'total_byes', 'total_legbyes', 'total_runs_per_innings_match',
                       'last_ball', 'total_wickets', 'date']

result_df['total_overs_played'] = result_df['last_ball'].apply(lambda x: min((int(x) + round((x % 1) * 10, 4) / 6), 50.0))

result_df[['total_wides', 'total_noballs', 'total_byes', 'total_legbyes']] = result_df[['total_wides', 'total_noballs', 'total_byes', 'total_legbyes']].astype(int)

result_df['total_runs_per_innings_match'] += result_df['total_wides'] + result_df['total_noballs'] + result_df['total_byes'] + result_df['total_legbyes']

result_df.drop(['last_ball'], axis=1, inplace=True)

In [4]:
result_df.head()

,match_id,innings,venue,batting_team,bowling_team,total_wides,total_noballs,total_byes,total_legbyes,total_runs_per_innings_match,total_wickets,date,total_overs_played
0,1,1,"Narendra Modi Stadium, Ahmedabad",England,New Zealand,6,0,0,0,282,9,2023-10-05,50.000000
1,1,2,"Narendra Modi Stadium, Ahmedabad",New Zealand,England,3,0,4,1,283,1,2023-10-05,36.333333
2,2,1,"Rajiv Gandhi International Stadium, Uppal, Hyd...",Pakistan,Netherlands,8,1,0,0,286,10,2023-10-06,49.000000
3,2,2,"Rajiv Gandhi International Stadium, Uppal, Hyd...",Netherlands,Pakistan,9,0,0,0,205,10,2023-10-06,41.166667
4,3,1,"Himachal Pradesh Cricket Association Stadium, ...",Afghanistan,Bangladesh,8,0,0,0,156,10,2023-10-07,37.333333


In [5]:
result_df.drop(['total_wickets','date','match_id','total_wides', 'total_noballs', 'total_byes', 'total_legbyes'], axis=1, inplace=True)
result_df.head()

,innings,venue,batting_team,bowling_team,total_runs_per_innings_match,total_overs_played
0,1,"Narendra Modi Stadium, Ahmedabad",England,New Zealand,282,50.000000
1,2,"Narendra Modi Stadium, Ahmedabad",New Zealand,England,283,36.333333
2,1,"Rajiv Gandhi International Stadium, Uppal, Hyd...",Pakistan,Netherlands,286,49.000000
3,2,"Rajiv Gandhi International Stadium, Uppal, Hyd...",Netherlands,Pakistan,205,41.166667
4,1,"Himachal Pradesh Cricket Association Stadium, ...",Afghanistan,Bangladesh,156,37.333333


In [6]:
final_df = result_df

In [7]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

mapping = {}
categorical_columns = ['venue','batting_team','bowling_team']

for column in categorical_columns:
    final_df[column] = le.fit_transform(final_df[column])
    mapping[column] = dict(zip(le.classes_, le.transform(le.classes_)))

In [8]:
X=final_df.drop('total_overs_played',axis='columns')
y=final_df['total_overs_played']

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

size_scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = size_scaler.transform(X_train)
X_test_scaled = size_scaler.transform(X_test)
X_train_scaled.shape, X_test_scaled.shape

((51, 5), (13, 5))

In [10]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.576831787244724

In [11]:
y_pred = model.predict(X_test)
y_pred

array([55.56297797, 44.13511808, 48.3471249 , 53.79311652, 36.91184664,
       45.15328175, 48.21465384, 53.71272828, 40.00902533, 38.00549489,
       44.24982331, 39.14811459, 46.55150632])

In [12]:
y_test

52    49.333333
58    49.666667
0     50.000000
44    50.000000
5     34.666667
36    49.833333
16    50.000000
12    50.000000
25    40.500000
61    32.500000
56    50.000000
9     41.333333
40    50.000000
Name: total_overs_played, dtype: float64

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler


model = keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),  # Input layer
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(1, activation='linear')  # Output layer with 1 unit and linear activation
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10)

model.fit(X_train, y_train, epochs=500, batch_size=1, verbose=1, callbacks=[early_stopping], validation_split=0.2)

Epoch 1/500
40/40 [==============================] - 1s 5ms/step - loss: 1371.3958 - mae: 33.1024 - val_loss: 81.6535 - val_mae: 7.7938
Epoch 2/500
40/40 [==============================] - 0s 2ms/step - loss: 97.0464 - mae: 7.8131 - val_loss: 61.7584 - val_mae: 6.4599
Epoch 3/500
40/40 [==============================] - 0s 2ms/step - loss: 90.1286 - mae: 8.1518 - val_loss: 59.9831 - val_mae: 6.4091
Epoch 4/500
40/40 [==============================] - 0s 2ms/step - loss: 89.6972 - mae: 7.9727 - val_loss: 59.9055 - val_mae: 6.3961
Epoch 5/500
40/40 [==============================] - 0s 2ms/step - loss: 89.5585 - mae: 7.7943 - val_loss: 60.6881 - val_mae: 6.4044
Epoch 6/500
40/40 [==============================] - 0s 2ms/step - loss: 94.8641 - mae: 8.3993 - val_loss: 71.5755 - val_mae: 6.5532
Epoch 7/500
40/40 [==============================] - 0s 2ms/step - loss: 88.5574 - mae: 7.8443 - val_loss: 59.2757 - val_mae: 6.3446
Epoch 8/500
40/40 [==============================] - 0s 2ms/step -

In [14]:
train_loss, train_mae = model.evaluate(X_train, y_train, verbose=1)
print(f"Training MAE: {train_mae}")

# Evaluate on the test set
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=1)
print(f"Test MAE: {test_mae}")

2/2 [==============================] - 0s 19ms/step - loss: 57.9438 - mae: 6.3452
Training MAE: 6.345240116119385
1/1 [==============================] - 0s 14ms/step - loss: 65.1814 - mae: 7.3659
Test MAE: 7.3659234046936035


In [15]:
import pickle as pkl

In [16]:
pkl.dump(model, open('../pickles/over_predictor', 'wb'))